In [1]:
import pandas as pd
import numpy as np
import os
from helper_functions import *

### Loading and Cleaning Datasets

In [2]:
pop = pd.read_excel('data/clusterBD.xlsx', sheet_name='population')
hf = pd.read_excel('data/clusterBD.xlsx', sheet_name='W5')
print(f'Shape of pop dataframe: {pop.shape}\nShape of hf dataframe: {hf.shape}')

Shape of pop dataframe: (765, 6)
Shape of hf dataframe: (11621, 27)


In [3]:
hf = clean_strings(hf, ['REGION', 'DISTRICT-SANITAIRE', 'AIRE-SANTE', 'SIGLE'])
hf = hf[hf['DISTRICT-SANITAIRE'].notna()]
hf = hf.drop('datesaisie', axis=1)

In [4]:
hf.loc[hf['DISTRICT-SANITAIRE'] == 'COMMUNE 1', 'DISTRICT-SANITAIRE'] = 'COMMUNE I'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'COMMUNE 2', 'DISTRICT-SANITAIRE'] = 'COMMUNE II'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'COMMUNE 3', 'DISTRICT-SANITAIRE'] = 'COMMUNE III'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'COMMUNE 4', 'DISTRICT-SANITAIRE'] = 'COMMUNE IV'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'COMMUNE 5', 'DISTRICT-SANITAIRE'] = 'COMMUNE V'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'COMMUNE 6', 'DISTRICT-SANITAIRE'] = 'COMMUNE VI'
pop.loc[pop.cercle == 'DJENNEE', 'cercle'] = 'DJENNE'

In [5]:
hf.loc[hf['DISTRICT-SANITAIRE'] == 'ACHOURATT', 'DISTRICT-SANITAIRE'] = 'ACHOURAT'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'ANSONGO ', 'DISTRICT-SANITAIRE'] = 'ANSONGO'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'ALMOUSTRAT', 'DISTRICT-SANITAIRE'] = 'ALMOUSTARAT'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'BAFOULABÉ', 'DISTRICT-SANITAIRE'] = 'BAFOULABE'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'ARAWANE', 'DISTRICT-SANITAIRE'] = 'ARAOUANE'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'BANAMBA ', 'DISTRICT-SANITAIRE'] = 'BANAMBA'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'BANKASS ', 'DISTRICT-SANITAIRE'] = 'BANKASS'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'BARAOUELI', 'DISTRICT-SANITAIRE'] = 'BAROUELI'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'DIRÉ', 'DISTRICT-SANITAIRE'] = 'DIRE'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'DIÉMA', 'DISTRICT-SANITAIRE'] = 'DIEMA'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'GOURMA RHAROUS', 'DISTRICT-SANITAIRE'] = 'GOURMA-RHAROUS'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'MÉNAKA, ANDÉRAMBOUKANE, TIDERMÈNE ET INÉKAR', 'DISTRICT-SANITAIRE'] = 'MENAKA'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'NIAFUNKÉ', 'DISTRICT-SANITAIRE'] = 'NIAFUNKE'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'OUSSOUBIDIAGNAN', 'DISTRICT-SANITAIRE'] = 'OUSSOUBIDIAGNA'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'SÉGOU', 'DISTRICT-SANITAIRE'] = 'SEGOU'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'YÉLIMAMÉ', 'DISTRICT-SANITAIRE'] = 'YELIMANE'
hf.loc[hf['DISTRICT-SANITAIRE'] == 'YOUWAROU ', 'DISTRICT-SANITAIRE'] = 'YOUWAROU'

In [6]:
hf.loc[hf.TYPOLOGIE == 'ONG nationale', 'TYPOLOGIE'] = 'ONG Nationale'

In [7]:
hf.loc[hf['SIGLE'] == ' ASDAP\xa0', 'SIGLE'] = 'ASDAP'
hf.loc[hf['SIGLE'] == 'CICR ', 'SIGLE'] = 'CICR'
hf.loc[hf['SIGLE'] == 'ENDA- MALI', 'SIGLE'] = 'ENDA-MALI'
hf.loc[hf['SIGLE'] == 'GAAS MALI', 'SIGLE'] = 'GAAS-MALI'
hf.loc[hf['SIGLE'] == 'IEDA RELIEF', 'SIGLE'] = 'IEDA-RELIEF'
hf.loc[hf['SIGLE'] == 'MDM BE', 'SIGLE'] = 'MDMBE'
hf.loc[hf['SIGLE'] == 'MSF E', 'SIGLE'] = 'MSF-E'
hf.loc[hf['SIGLE'] == 'PU-I', 'SIGLE'] = 'PUI'
hf.loc[hf['SIGLE'] == 'SANTÉ  SUD', 'SIGLE'] = 'SANTE SUD'
hf.loc[hf['SIGLE'] == 'USAID/HRH 2030', 'SIGLE'] = 'USAID/HRH2030'
hf.loc[hf['SIGLE'] == 'PU-I', 'SIGLE'] = 'PUI'

### Administrative Areas and Population

In [8]:
regions = convert_to_csv(hf.REGION.unique(), 'REGION', 'regions')
regions = get_serial_column(regions, ['REGION'], 'RegionIndex')

In [9]:
district_sanitaire = regions.merge(hf[['REGION', 'DISTRICT-SANITAIRE']], on='REGION', how='inner')
district_sanitaire = district_sanitaire.drop_duplicates('DISTRICT-SANITAIRE', keep='last')
district_sanitaire = district_sanitaire.sort_values(by=['RegionIndex', 'DISTRICT-SANITAIRE'])
district_sanitaire = district_sanitaire.rename(columns = {'DISTRICT-SANITAIRE': 'CERCLE'})
district_sanitaire.drop(['REGION'], axis=1, inplace=True)
district_sanitaire = district_sanitaire[['CERCLE', 'RegionIndex']]
district_sanitaire.to_csv("data/cercle.csv", index = False)

In [10]:
district_sanitaire = get_serial_column(district_sanitaire, ['RegionIndex', 'CERCLE'], 'CercleIndex')

In [11]:
commune = district_sanitaire.merge(pop[['cercle', 'commune']], left_on='CERCLE', right_on='cercle', how='inner')
assert commune.CERCLE.all() == commune.cercle.all()
commune_copy = commune.copy()
commune = commune.drop(['cercle', 'CERCLE'], axis=1)
commune = commune.rename(columns = {'commune': 'COMMUNE'})
commune = commune[['COMMUNE', 'CercleIndex', 'RegionIndex']]
commune.to_csv("data/commune.csv", index = False)

In [12]:
commune_copy.drop(['CERCLE'], axis=1, inplace=True)
population = commune_copy.merge(pop[['cercle', 'commune', 'masculin', 'feminin', 'datesaisie']], on=['cercle', 'commune'])
population.reset_index(inplace=True)
population['index'] += 1
population = population.rename(columns = {'index': 'CommuneIndex'})
population = population.drop(['cercle', 'commune'], axis=1)
population = population[['RegionIndex', 'CercleIndex', 'CommuneIndex', 'masculin', 'feminin', 'datesaisie']]
population.to_csv("data/population.csv", index = False)

In [13]:
assert population.shape == pop.shape
assert population.CommuneIndex.nunique() == pop.shape[0]

### W5

In [14]:
types = convert_to_csv(hf.TYPOLOGIE.unique(), 'TYPOLOGIE', 'typology')
sigle = convert_to_csv(hf.SIGLE.unique(), 'SIGLE', 'sigle')

In [15]:
types = get_serial_column(types, ['TYPOLOGIE'], 'TypeIndex')
sigle = get_serial_column(sigle, ['SIGLE'], 'SigleIndex')

In [16]:
hf.loc[hf['AIRE-SANTE'].isnull(), 'AIRE-SANTE'] = 'NOT AVAILABLE'

In [17]:
aire_sante = district_sanitaire.merge(hf[['DISTRICT-SANITAIRE', 'AIRE-SANTE']], left_on='CERCLE', right_on='DISTRICT-SANITAIRE', how='inner')
assert aire_sante['DISTRICT-SANITAIRE'].all() == aire_sante.CERCLE.all()
aire_sante = aire_sante[['AIRE-SANTE', 'CercleIndex', 'RegionIndex']]
aire_sante = aire_sante.groupby(['AIRE-SANTE', 'CercleIndex', 'RegionIndex']).size().reset_index(name='Freq')
aire_sante = aire_sante.sort_values(by=['RegionIndex', 'CercleIndex', 'AIRE-SANTE'], axis=0)
aire_sante = aire_sante.drop(['Freq'], axis=1)
aire_sante.to_csv("data/aire_sante.csv", index = False)

In [18]:
aire_sante = aire_sante.reset_index().reset_index()
aire_sante.drop(['index'], axis=1, inplace=True)
aire_sante.level_0 += 1
aire_sante = aire_sante.rename(columns={'level_0': 'AireIndex'})

In [19]:
indexed_hf = merge_columns(hf, regions, ['REGION'], ['REGION'], ['REGION'])
indexed_hf = indexed_hf.rename(columns={'index': 'RegionIndex'})

In [20]:
indexed_hf = merge_columns(indexed_hf, district_sanitaire, ['RegionIndex', 'DISTRICT-SANITAIRE'], ['RegionIndex', 'CERCLE'], ['CERCLE', 'DISTRICT-SANITAIRE'])
indexed_hf = merge_columns(indexed_hf, aire_sante, ['RegionIndex', 'CercleIndex', 'AIRE-SANTE'], ['RegionIndex', 'CercleIndex', 'AIRE-SANTE'], ['AIRE-SANTE'])
indexed_hf = indexed_hf.rename(columns = {'RegionIndex': 'REGION', 'CercleIndex': 'DISTRICT-SANITAIRE', 'AireIndex': 'AIRE-SANTE'})
assert indexed_hf.shape == hf.shape

In [21]:
old_new = [(0, 1), (1, 2), ('x', 3), ('X', 4), ('✓', 5), ('***', 6), ('PSA GAINA  BOSSOBON', 7), ('PSA KAREIBANDIAKA', 8), ('PSA INASSAKOK', 9), ('x (Nexus)', 10)]
columns = ['PSA', 'EQUIPE MOBILE', 'URENAM', 'URENAS', 'URENI', 'Urgence', 'Transition', 'Developpement']
indexed_hf = replace_values(indexed_hf, columns, old_new)

In [22]:
functional = []
for i in old_new:
    functional.append(i[0])
services = pd.Series(functional).to_frame()
services.reset_index(inplace=True)
services = services.rename(columns = {0: 'Functional'})
services.drop('index', axis=1, inplace=True)
services.to_csv(f"data/services_functional.csv", index = False)

In [23]:
list_fonctionnel = ['fonctionnel', ' fonctionnel', 'Foncionnel', 'F', 'Fonctionnel ( CSRef)', 'Fonctionnel ']
list_non_fonctionnel = ['Non fonctionnel', 'NF', 'Non-fonctionnel', 'x']
indexed_hf['Etat'] = indexed_hf['Etat'].replace(list_fonctionnel, 'Fonctionnel')
indexed_hf['Etat'] = indexed_hf['Etat'].replace(list_non_fonctionnel, 'Non Fonctionnel')
indexed_hf['Etat'] = indexed_hf['Etat'].replace(['OUI', 'oui'], 'Oui')
indexed_hf['Etat'] = indexed_hf['Etat'].replace('non', 'Non')
indexed_hf['Etat'] = indexed_hf['Etat'].replace(np.nan, 'Missing')

In [24]:
etat = convert_to_csv(indexed_hf.Etat.unique(), 'Etat', 'etat')
etat = get_serial_column(etat, ['Etat'], 'EtatIndex')

In [25]:
indexed_hf = merge_columns(indexed_hf, etat, ['Etat'], ['Etat'], ['Etat'])
indexed_hf = merge_columns(indexed_hf, types, ['TYPOLOGIE'], ['TYPOLOGIE'], ['TYPOLOGIE'])
indexed_hf = merge_columns(indexed_hf, sigle, ['SIGLE'], ['SIGLE'], ['SIGLE'])

In [26]:
indexed_hf = indexed_hf.rename(columns={'EtatIndex': 'Etat', 'TypeIndex': 'TYPOLOGIE', 'SigleIndex': 'SIGLE'})

In [27]:
reordered_columns = ['ID', 'TYPOLOGIE', 'NOM', 'SIGLE', 'REGION', 'DISTRICT-SANITAIRE', 'AIRE-SANTE', 'HOPITAL', 'CSREF', 'CSCOM', 'PSA', 'EQUIPE MOBILE', 'SOINS-DE', 'URENAM', 'URENAS', 'URENI', 'Population', 'Latitude(X)',
                     'Longitude(Y)', 'Etat', 'Urgence', 'Transition', 'Developpement', 'debut', 'fin', 'Financements']

In [28]:
indexed_hf = indexed_hf.loc[:, reordered_columns]
indexed_hf = indexed_hf.sort_values(by='ID', axis=0)

In [29]:
indexed_hf.to_csv("data/service_availability.csv", index = False)